# Predicting Smart Grid Stability

<p style="text-align: justify">This notebook is based on the "<b>Electrical Grid Stability Simulated Dataset</b>", created by Vadim Arzamasov (Karlsruher Institut für Technologie, Karlsruhe, Germany) and donated to the <b>University of California (UCI) Machine Learning Repository</b> (link <a href="https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+#">here</a>), where it is currently hosted.</p> Learning from the project (link <a 
href="https://www.kaggle.com/code/pcbreviglieri/predicting-smart-grid-stability-with-deep-learning">here</a>)
we evaluate the different algorithms with the same dataset here to do more analysis in deep learning.

The Smart Grid system is described as follows with three consumption nodes and one renewable generation node.

<img src="https://i.imgur.com/hvmW0cg.png" width="500" height="100">

## Objectives of this exercise

The Main objective of the analysis is to focus on a specific type of Deep Learning and the benefits that this analysis brings to the business or stakeholders of this data is to predict the stability of grid more accurately.

## Brief description of the data set 

<p style="text-align: justify">The dataset chosen for this machine learning exercise has a synthetic nature and contains results from simulations of grid stability for a reference 4-node star network.</p>
<p style="text-align: justify">The original dataset contains 10,000 observations. As the reference grid is symetric, the dataset can be augmented in 3! (3 factorial) times, or 6 times, representing a permutation of the three consumers occupying three consumer nodes. The augmented version has then <b>60,000 observations</b>. It also contains <b>12 primary predictive features</b> and two dependent variables. </p>
<p><b>Predictive features</b>:</p>
<ol>
    <li>'tau1' to 'tau4': the reaction time of each network participant, a real value within the range 0.5 to 10 ('tau1' corresponds to the supplier node, 'tau2' to 'tau4' to the consumer nodes);</li>
    <li>'p1' to 'p4': nominal power produced (positive) or consumed (negative) by each network participant, a real value within the range -2.0 to -0.5 for consumers ('p2' to 'p4'). As the total power consumed equals the total power generated, p1 (supplier node) = - (p2 + p3 + p4);</li>
    <li>'g1' to 'g4': price elasticity coefficient for each network participant, a real value within the range 0.05 to 1.00 ('g1' corresponds to the supplier node, 'g2' to 'g4' to the consumer nodes; 'g' stands for 'gamma');</li>
</ol>
<p><b>Dependent variables</b>:</p>
<ol>
    <li>'stab': the maximum real part of the characteristic differentia equation root (if positive, the system is linearly unstable; if negative, linearly stable);</li>
    <li>'stabf': a categorical (binary) label ('stable' or 'unstable').</li>
</ol>
<p style="text-align: justify">As there is a direct relationship between 'stab' and 'stabf' ('stabf' = 'stable' if 'stab' <= 0, 'unstable' otherwise), 'stab' will be dropped and 'stabf' will remain as the sole dependent variable.</p>
<p style="text-align: justify">As the dataset content comes from simulation exercises, there are no missing values. Also, all features are originally numerical, no feature coding is required. Such dataset properties allow for a direct jump to machine modeling without the need of data preprocessing or feature engineering.</p>

## Exploratory  Data Analysis

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score

## Import Keras objects for Deep Learning
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

from datetime import datetime

<p style="text-align: justify">The augmented dataset (60,000 observations) is imported.</p>

In [49]:
sns.set()
start_time = datetime.now()

data = pd.read_csv('./smart_grid_stability_augmented.csv')


In [50]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    60000 non-null  float64
 1   tau2    60000 non-null  float64
 2   tau3    60000 non-null  float64
 3   tau4    60000 non-null  float64
 4   p1      60000 non-null  float64
 5   p2      60000 non-null  float64
 6   p3      60000 non-null  float64
 7   p4      60000 non-null  float64
 8   g1      60000 non-null  float64
 9   g2      60000 non-null  float64
 10  g3      60000 non-null  float64
 11  g4      60000 non-null  float64
 12  stab    60000 non-null  float64
 13  stabf   60000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 6.4+ MB


The dependent variable is map encoded ('stable' replaced with 1, 'unstable' with 0). At last, the 60,000 observations are shuffled.

In [52]:
map1 = {'unstable': 0, 'stable': 1}
data['stabf'] = data['stabf'].replace(map1)

data = data.sample(frac=1)

In [53]:
data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
1576,7.589726,7.334232,3.888468,1.614712,3.414727,-0.887893,-1.240917,-1.285917,0.235765,0.678952,0.183857,0.506624,-0.009720,1
22563,0.672029,3.740985,8.657039,5.808404,3.851821,-1.140452,-1.175718,-1.535652,0.560357,0.275093,0.564756,0.190414,-0.034273,1
56945,6.913046,3.738946,2.015020,0.725520,2.960418,-1.729515,-0.642648,-0.588255,0.232532,0.862134,0.620180,0.691592,-0.039918,1
54750,4.585690,4.133391,8.733741,2.433056,4.545806,-0.760712,-1.795753,-1.989341,0.749258,0.769378,0.051526,0.568069,0.029752,0
43664,6.308387,0.546782,8.967333,6.480536,3.655075,-1.304979,-1.750637,-0.599459,0.328626,0.159850,0.684227,0.488982,0.023936,0


In [54]:
data.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000
mean,5.250000,5.250001,5.250001,5.250001,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731,0.362000
std,2.742434,2.742437,2.742437,2.742437,0.752129,0.433017,0.433017,0.433017,0.274244,0.274243,0.274243,0.274243,0.036917,0.480583
min,0.500793,0.500141,0.500141,0.500141,1.582590,-1.999945,-1.999945,-1.999945,0.050009,0.050028,0.050028,0.050028,-0.080760,0.000000
25%,2.874892,2.875011,2.875011,2.875011,3.218300,-1.624997,-1.624997,-1.624997,0.287521,0.287497,0.287497,0.287497,-0.015557,0.000000
50%,5.250004,5.249981,5.249981,5.249981,3.751025,-1.249996,-1.249996,-1.249996,0.525009,0.525007,0.525007,0.525007,0.017142,0.000000
75%,7.624690,7.624896,7.624896,7.624896,4.282420,-0.874993,-0.874993,-0.874993,0.762435,0.762490,0.762490,0.762490,0.044878,1.000000
max,9.999469,9.999837,9.999837,9.999837,5.864418,-0.500025,-0.500025,-0.500025,0.999937,0.999982,0.999982,0.999982,0.109403,1.000000


In [55]:
data.p1.skew()

-0.012688423269883422

In [56]:
data.shape

(60000, 14)

<p style="text-align: justify">The proportion of observations related to 'unstable' and 'stable' scenarios is mapped.</p>

In [57]:
print(f'Split of "unstable" (0) and "stable" (1) observations in the original dataset:')
print(data['stabf'].value_counts(normalize=True))

Split of "unstable" (0) and "stable" (1) observations in the original dataset:
0    0.638
1    0.362
Name: stabf, dtype: float64


It is important to verify the correlation between each numerical feature and the dependent variable.

In [58]:
data.corr()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
tau1,1.000000,-0.002550,-0.002550,-0.002550,0.027183,-0.015739,-0.015739,-0.015739,0.010521,0.006522,0.006522,0.006522,0.275761,-0.234898
tau2,-0.002550,1.000000,0.005554,0.005554,0.003004,-0.004473,-0.000372,-0.000372,-0.005832,0.009865,0.002102,0.002102,0.283417,-0.241049
tau3,-0.002550,0.005554,1.000000,0.005554,0.003004,-0.000372,-0.004473,-0.000372,-0.005832,0.002102,0.009865,0.002102,0.283417,-0.241049
tau4,-0.002550,0.005554,0.005554,1.000000,0.003004,-0.000372,-0.000372,-0.004473,-0.005832,0.002102,0.002102,0.009865,0.283417,-0.241049
p1,0.027183,0.003004,0.003004,0.003004,1.000000,-0.578983,-0.578983,-0.578983,0.000721,0.000341,0.000341,0.000341,0.010278,-0.009938
p2,-0.015739,-0.004473,-0.000372,-0.000372,-0.578983,1.000000,0.002833,0.002833,-0.000417,-0.002141,0.000774,0.000774,-0.005951,0.005754
p3,-0.015739,-0.000372,-0.004473,-0.000372,-0.578983,0.002833,1.000000,0.002833,-0.000417,0.000774,-0.002141,0.000774,-0.005951,0.005754
p4,-0.015739,-0.000372,-0.000372,-0.004473,-0.578983,0.002833,0.002833,1.000000,-0.000417,0.000774,0.000774,-0.002141,-0.005951,0.005754
g1,0.010521,-0.005832,-0.005832,-0.005832,0.000721,-0.000417,-0.000417,-0.000417,1.000000,0.004718,0.004718,0.004718,0.282774,-0.197664
g2,0.006522,0.009865,0.002102,0.002102,0.000341,-0.002141,0.000774,0.000774,0.004718,1.000000,-0.006939,-0.006939,0.293684,-0.218015


### Spliting train and test sets

<p style="text-align: justify">As anticipated, the features dataset will contain all 12 original predictive features, while the label dataset will contain only 'stabf' ('stab' is dropped here).</p>
<p style="text-align: justify">In addition, as the dataset has already been shuffled, the training set will receive the first 54,000 observations, while the testing set will accommodate the last 6,000.</p>

In [9]:
X = data.iloc[:, :12]
y = data["stabf"].values

# Split the data to Train, and Test (90%, 10%)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [10]:
np.mean(y), np.mean(1-y)

(0.362, 0.638)

In [11]:
X_train.shape

(54000, 12)

In [12]:
y_train.shape

(54000,)

### Feature scaling

<p style="text-align: justify">In preparation for machine learning, scaling is performed based on (fitted to) the training set and applied (with the 'transform' method) to both training and testing sets.</p>

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
X_train

array([[-1.19086778e-01, -1.56163839e+00,  9.89381432e-01, ...,
         3.04953306e-02, -1.46751237e+00,  1.80204198e-02],
       [ 1.71020882e+00,  1.12089273e+00,  1.64729285e+00, ...,
         6.49968217e-01, -1.17603727e-01, -5.87522188e-01],
       [-5.19021295e-01,  1.58353009e-03, -8.78156107e-01, ...,
         5.17566964e-01, -1.68899538e+00,  1.68723151e+00],
       ...,
       [ 1.04235634e+00,  5.34161694e-01, -2.90534288e-01, ...,
        -1.41278635e+00,  7.55331083e-01,  1.63123941e+00],
       [ 2.73849212e-01, -9.07753807e-01, -1.08272553e-01, ...,
         1.26218742e+00, -1.40593482e+00,  1.72301739e+00],
       [ 6.85063076e-01,  1.10548771e+00,  7.20730022e-01, ...,
         1.45735783e+00, -9.67856217e-01,  1.29320036e+00]])

## Deep Learning
Define the Model 
Input size is 12-dimensional
1 hidden layer, 24 hidden nodes, sigmoid activation
Final layer has just one node with a sigmoid activation (standard for binary classification)

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
Input_dimensions = X.shape[1]
def model_creation(layers,activation):
    dl_model = Sequential()
    for l,nodes in enumerate(layers):
        if l==0:
            dl_model.add(Dense(nodes,input_dim =Input_dimensions,activation = activation))
        else:
            dl_model.add(Dense(nodes,activation = activation))
          
    dl_model.add(Dense(1,activation='sigmoid'))
    
    dl_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return dl_model

dl_model = KerasClassifier(build_fn=model_creation, verbose =0)
            

In [35]:
layers = [[24], [24,24], [24,24,12], [24,24,12,12]]
activations = ['sigmoid','relu']
param_grid = dict(layers = layers,activation=activations, batch_size = [128,256], epochs=[50])
grid = GridSearchCV(estimator=dl_model,param_grid=param_grid)

In [36]:
results_grid = grid.fit(X_train,y_train)

In [37]:
[results_grid.best_score_,results_grid.best_params_]

[0.9754999995231628,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 50,
  'layers': [24, 24, 12, 12]}]

In [38]:
y_pred = grid.predict(X_test)
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

In [39]:
cm = pd.DataFrame(data=confusion_matrix(y_test, y_pred, labels=[0, 1]),
                  index=["Actual Unstable", "Actual Stable"],
                  columns=["Predicted Unstable", "Predicted Stable"])
cm

,Predicted Unstable,Predicted Stable
Actual Unstable,3827,31
Actual Stable,146,1996


In [40]:
print(f'Accuracy per the confusion matrix: {((cm.iloc[0, 0] + cm.iloc[1, 1]) / len(y_test) * 100):.2f}%')

Accuracy per the confusion matrix: 97.05%


## Discussion

This exercise provides a better understanding of deep learning approach in case of predicting grid stability. We used GridSearchCV library to get the best parameters for the model such as layers and nodes and activation function. We got 97.05% accuracy of the model. From hyperparameter tuning we got four layers with ReLu activation. 
For further analysis with this dataset we can increase the number of epochs and it will increase the prediction accuracy.


In [41]:
end_time = datetime.now()

print('\nStart time', start_time)
print('End time', end_time)
print('Time elapsed', end_time - start_time)


Start time 2022-09-13 11:41:37.561602
End time 2022-09-13 12:28:46.350306
Time elapsed 0:47:08.788704
